In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import os
import pandas as pd

from utils.makepath import makepath

In [18]:
root_dir = makepath("..", "..")

os.listdir(root_dir)

['paper',
 'gradops',
 '.DS_Store',
 'activate_venv.sh',
 'LICENSE',
 'requirements.txt',
 '.pytest_cache',
 'config',
 'pyproject.toml',
 'tests',
 'utils',
 'docs',
 'TODO.md',
 '.readthedocs.yml',
 'README.md',
 'results',
 'networks',
 'data_lib',
 '.gitignore',
 'encoding_objects',
 'figures',
 'scripts',
 '.gitattributes',
 'train_mri_example.ipynb',
 'report',
 'venv',
 '.git',
 'pdhg',
 'data',
 'results copy',
 'tmp',
 'notebooks',
 'pretrained',
 'mri.egg-info']

In [19]:
results_path = makepath(root_dir, "results", "test_set")

os.listdir(results_path)

['u_tv',
 'u_tv_fixed',
 '.DS_Store',
 'new_u_tgv',
 'zero_filled',
 'scalar_tv',
 'u_tgv_fixed',
 'u_tgv',
 'new_u_tv',
 'scalar_tgv']

## Scalar manual search

In [20]:
# model = "scalar_tgv"
model = "scalar_tv"
best_metric = "SSIM"
for acc_R in [4, 8]:
    for sigma in ["0_05", "0_1", "0_2"]:
        # acc_R, sigma, datetime = 4, "0_05", "21_01_59"
        # acc_R, sigma, datetime = 4, "0_1", "21_03_20"
        # csv_file = f"test-abs-scalar_tgv-best_SSIM-R_{acc_R}-sigma_{sigma}-lambda0_w_0_to_0_2-lambda1_v_0_to_0_2-ternary_iters_20-dt_{datetime}.csv"
        for file in os.listdir(makepath(results_path, model)):
            if f"test-abs-{model}-best_{best_metric}-R_{acc_R}-sigma_{sigma}" in file and file.endswith(".csv"):
                csv_file = file
                break
        df = pd.read_csv(makepath(results_path, model, csv_file))
        # print(df.head())
        print(f"R = {acc_R}, sigma = {sigma}")
        print(f"PSNR: {df['PSNR'].mean():.2f} ± {df['PSNR'].std():.2f}")
        print(f"SSIM: {df['SSIM'].mean():.4f} ± {df['SSIM'].std():.3f}")
        print()

R = 4, sigma = 0_05
PSNR: 27.21 ± 1.81
SSIM: 0.7134 ± 0.088

R = 4, sigma = 0_1
PSNR: 27.15 ± 1.82
SSIM: 0.7144 ± 0.090

R = 4, sigma = 0_2
PSNR: 26.91 ± 1.79
SSIM: 0.7096 ± 0.088

R = 8, sigma = 0_05
PSNR: 23.92 ± 1.71
SSIM: 0.6262 ± 0.085

R = 8, sigma = 0_1
PSNR: 23.87 ± 1.71
SSIM: 0.6264 ± 0.085

R = 8, sigma = 0_2
PSNR: 23.77 ± 1.71
SSIM: 0.6249 ± 0.085



In [21]:
csv_path = makepath(results_path, model, csv_file)

df = pd.read_csv(csv_path)
df.head()

,id,PSNR,SSIM,lambda
0,1,24.973553,0.685744,0.014551
1,2,23.652971,0.560019,0.189642
2,3,26.778214,0.603287,0.155243
3,4,23.680239,0.672584,0.012823
4,5,22.243095,0.625197,0.022588


In [22]:
df.mean()

id        151.500000
PSNR       23.771406
SSIM        0.624861
lambda      0.069194
dtype: float64

In [23]:
df.std()

id        87.324109
PSNR       1.712432
SSIM       0.085098
lambda     0.061887
dtype: float64

## Pre-trained models

In [24]:
# model = "u_tgv"
# model = "new_u_tgv"
# model = "u_tgv_fixed"
model = "u_tv_fixed"
df_results = pd.DataFrame()
for acc_R in [4, 8]:
    for sigma in ["0_05", "0_10", "0_20"]:
        csv_file = makepath(
            # f"test-abs-R_{acc_R}-sigma_{sigma}",
            f"test-abs-R_{acc_R}-sigma_{sigma}_intermediate_metrics.csv",
        )
        df = pd.read_csv(makepath(results_path, model, csv_file))
        # print(df.head())
        print(f"R = {acc_R}, sigma = {sigma}")
        print(f"PSNR: {df['psnr'].mean():.2f} ± {df['psnr'].std():.2f}")
        print(f"SSIM: {df['ssim'].mean():.4f} ± {df['ssim'].std():.3f}")
        print()
        df_results = pd.concat([
            df_results,
            pd.DataFrame({
                "R": [acc_R],
                "sigma": [sigma],
                "PSNR": [f"{df['psnr'].mean():.2f} ± {df['psnr'].std():.2f}"],
                "SSIM": [f"{df['ssim'].mean():.4f} ± {df['ssim'].std():.3f}"],
            }),
        ])

R = 4, sigma = 0_05
PSNR: 32.60 ± 2.30
SSIM: 0.7129 ± 0.091

R = 4, sigma = 0_10
PSNR: 32.01 ± 2.20
SSIM: 0.7039 ± 0.093

R = 4, sigma = 0_20
PSNR: 30.94 ± 2.11
SSIM: 0.6976 ± 0.095

R = 8, sigma = 0_05
PSNR: 29.65 ± 2.25
SSIM: 0.6681 ± 0.095

R = 8, sigma = 0_10
PSNR: 29.22 ± 2.19
SSIM: 0.6607 ± 0.095

R = 8, sigma = 0_20
PSNR: 28.38 ± 2.14
SSIM: 0.6608 ± 0.095



In [25]:
# !pwd

In [26]:
# !less ../../results/test_set/u_tgv_fixed/test-abs-R_4-sigma_0_05/test-abs-R_4-sigma_0_05_intermediate_metrics.csv

In [27]:
!ls

Practical3_working.ipynb                print_statistics.ipynb
colorbar                                prox_ops.py
convert_pretrained_cuda_to_cpu.ipynb    results
difference_histograms.ipynb             show_training_progress.ipynb
example.ipynb                           simple_unet.txt
generate_results.ipynb                  split_data.ipynb
generate_results_tgv_scalar_torch.ipynb test.ipynb
generate_test_data.ipynb                test_example.ipynb
grad_ops.py                             test_load_model.ipynb
grid_search_tv.ipynb                    train_example.ipynb
make_plots.ipynb                        unet.txt
model_all_info_example.pt               unet_2d.txt
pretrained                              visualize.ipynb


In [28]:
print(df_results.to_latex())

\begin{tabular}{lrlll}
\toprule
 & R & sigma & PSNR & SSIM \\
\midrule
0 & 4 & 0_05 & 32.60 ± 2.30 & 0.7129 ± 0.091 \\
0 & 4 & 0_10 & 32.01 ± 2.20 & 0.7039 ± 0.093 \\
0 & 4 & 0_20 & 30.94 ± 2.11 & 0.6976 ± 0.095 \\
0 & 8 & 0_05 & 29.65 ± 2.25 & 0.6681 ± 0.095 \\
0 & 8 & 0_10 & 29.22 ± 2.19 & 0.6607 ± 0.095 \\
0 & 8 & 0_20 & 28.38 ± 2.14 & 0.6608 ± 0.095 \\
\bottomrule
\end{tabular}



In [29]:
# model = "u_tv"
# model = "new_u_tv"
model = "u_tv_fixed"
for acc_R in [4]:
    for sigma in ["0_05"]:
        csv_file = makepath(
            f"test-abs-R_{acc_R}-sigma_{sigma}",
            f"test-abs-R_{acc_R}-sigma_{sigma}_intermediate_metrics.csv",
        )
        df = pd.read_csv(makepath(results_path, model, csv_file))
        # print(df.head())
        print(f"R = {acc_R}, sigma = {sigma}")
        print(f"PSNR: {df['psnr'].mean():.2f} ± {df['psnr'].std():.2f}")
        print(f"SSIM: {df['ssim'].mean():.4f} ± {df['ssim'].std():.3f}")
        print()

FileNotFoundError: [Errno 2] No such file or directory: '../../results/test_set/u_tv_fixed/test-abs-R_4-sigma_0_05/test-abs-R_4-sigma_0_05_intermediate_metrics.csv'